In [ ]:
import geemap
import ee

Map = geemap.Map()
Map

In [ ]:
# roi = geemap.shp_to_ee(r'E:/Data/roi.shp')
modis = ee.ImageCollection("MODIS/006/MOD09Q1").filterDate('2018', '2019')\
        .filter(ee.Filter.calendarRange(120, 240, 'day_of_year'))
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI')\
            .filterDate('2018', '2019')\
            .filter(ee.Filter.calendarRange(120, 240, 'day_of_year'))
modis_names = modis.aggregate_array('system:id').getInfo()
landsat_names = landsat8.aggregate_array('system:id').getInfo()

path = r'E:/Data/NDVI_JX/'

In [ ]:
Map.addLayer(roi, {}, 'rio')

In [ ]:
roi = Map.draw_last_feature

In [ ]:
modis_names

In [ ]:
import os

def clip(img):
    return img.clip(roi.geometry())
def modis_NDVI(img):
    img1 = img.clip(roi.geometry())
    img2 = img1.normalizedDifference(['sur_refl_b02', 'sur_refl_b01'])
    return img2
def export_image(i):
    modis_name = modis_names[i]
    landsat_name = landsat_names[i]
    modis_img = modis_NDVI(ee.Image(modis_name))
    landsat_img = clip(ee.Image(landsat_name))
    NDVI_img = landsat_img.unmask(modis_img)
    f_name = 'NDVI_'+modis_name.split('/')[-1]
    geemap.ee_export_image_to_drive(NDVI_img, description=f_name, crs='epsg:3857',
                                    folder='Image', scale=30)
#     return f_name+'.tif'
def Downlocal(List):
    global gauth
    global drive
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()
    drive = GoogleDrive(gauth)
    os.system('taskkill /F /IM msedge.exe')
    for f_list in List:
        out_file = path + f_list['title']
        f_list.GetContentFile(out_file)
        print(out_file+'下载成功！！')
        f_list.Delete()

In [ ]:
for i in range(len(modis_names)):
    export_image(i)
#     if (i % 5 == 4) or (i == len(modis_names)-1):
#         file_list = []
#         print('正在检测.....')
#         while(len(file_list) < i%5+1):
#             file_list = drive.ListFile({'q':"mimeType='image/tiff'"}).GetList()
#         if flag:
#             print('开始下载.....')
#             Downlocal(file_list)

In [ ]:
export_image(-3)

In [ ]:
import os
path = r'E:/Data/NDVI_JX/'
j = 30
k = 34
flag = True
while(flag):
    file_list = []
    while(len(file_list) < k-j):
        file_list = drive.ListFile({'q':"mimeType='image/tiff'"}).GetList()
    j += 5
    k += 5
    if k>len(modis_names):
        k = len(modis_names)
    for i in range(j, k):
        export_image(i)
    drive = GoogleDrive(gauth)
    os.system('taskkill /F /IM msedge.exe')
    for file in file_list:
        out_file = path + file['title']
        file.GetContentFile(out_file)
        print(out_file+'下载成功！！')
        file.Delete()
    if k > len(modis_names):
        break

In [ ]:
file_list = drive.ListFile({'q':"mimeType='image/tiff'"}).GetList()
Downlocal(file_list)

In [ ]:
import winsound
duration = 1000  # millisecond
freq = 440  # Hz
winsound.Beep(freq, duration)

In [ ]:
modis_names[40]

In [ ]:
file_list = drive.ListFile({'spaces':'user'}).GetList()

In [ ]:
file_list

In [ ]:
len(file_list)